# 🧑‍💻 Taller de Laboratorios de Machine Learning y Deep Learning

Integrantes:
- Anderson Bornachera
- Juan Mosquera

# Random Forest





## Cargamos el dataset a utilizar.

In [1]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

In [2]:
import pandas as pd

df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target
df.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## Caracteristicas de las columnas.

In [3]:
print(df.size)
print(df.shape)
df.info()
print("Nulls: ",df.isna().sum())

17639
(569, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness

## Dividir el dataset 80% para entrenamiento y 20% para test

In [4]:
from sklearn.model_selection import train_test_split

y = df['target']
X = df.drop(columns=['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Usando GridSearch buscamos los mejores hiperpárametros, y utilizamos el accuracy como métrica para obtener la mejor configuración

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    "n_estimators": [100, 200, 300],      # número de árboles
    "max_depth": [None, 5, 10, 20],       # profundidad máxima
    "min_samples_split": [2, 5, 10],      # mínimo de muestras para dividir un nodo
    "min_samples_leaf": [1, 2, 4],        # mínimo de muestras en hoja
    "max_features": ["sqrt", "log2"]      # número de features a considerar en cada split
}

rfc = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    cv=5,                # validación cruzada (5-fold)
    scoring="accuracy",  # métrica a optimizar
    n_jobs=-1,           # usar todos los núcleos
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor accuracy:", grid_search.best_score_)

# Evaluar en test
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Accuracy en test:", test_accuracy)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejores hiperparámetros: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mejor accuracy: 0.9604395604395606
Accuracy en test: 0.956140350877193


## Calculamos las métricas para poder  medir el desempeño del modelo.

In [6]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report)

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # para AUC-ROC


# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"Exactitud (Accuracy): {accuracy:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")
print("\nMatriz de confusión:\n", cm)

Exactitud (Accuracy): 0.9561
Precisión: 0.9589
Recall: 0.9722
F1: 0.9655
AUC-ROC: 0.9931

Matriz de confusión:
 [[39  3]
 [ 2 70]]


## Conclusión:

El modelo RandomForestClassifier obtuvo una alta exactitud y un AUC cercano a 1, lo que indica una excelente capacidad para diferenciar entre tumores malignos y benignos.

La precisión y el recall son elevados, mostrando que el modelo detecta correctamente la mayoría de los casos malignos sin generar demasiados falsos positivos.

La matriz de confusión confirma un bajo número de errores, especialmente en los casos malignos (lo más crítico en un contexto médico).

Entre las variables más importantes destacan mean radius, mean texture y worst perimeter, lo que coincide con criterios médicos usados para evaluar tumores.

# Red Neuronal (MLP conKeras)

## 1. Estandarización de variables numéricas con StandardScaler.

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 2. Implementación de **red neuronal secuencial** en TensorFlow Keras para un problema de clasificación binaria.  
- Se usan: `Sequential` para construir el modelo, `Dense` para capas densas, `Dropout` para reducir sobreajuste y `EarlyStopping` para detener el entrenamiento si no mejora.  
- La red tiene: una capa de 64 neuronas con `ReLU`, una capa `Dropout(0.2)`, otra capa de 32 neuronas con `ReLU` y una salida con 1 neurona y activación `sigmoid` (probabilidad entre 0 y 1).  
- Se compila con optimizador `adam`, pérdida `binary_crossentropy` y métricas `accuracy`, `Recall` y `Precision`.  

En resumen, es un modelo simple y efectivo para **clasificación binaria**, con regularización y métricas que permiten evaluar tanto la exactitud como la calidad de las predicciones positivas.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall', 'Precision'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


##  3. Entrenamiento con EarlyStopping.

**EarlyStopping** es usado para evitar sobreentrenamiento.
Se define `early_stopping` indicando que se debe monitorear la pérdida en validación (`val_loss`), con una tolerancia de 10 épocas sin mejora (`patience=10`) antes de detener el entrenamiento, y se restauran los mejores pesos obtenidos (`restore_best_weights=True`).

Luego, con `model.fit` se entrena el modelo usando los datos `X_train_scaled` y `y_train` durante un máximo de 100 épocas, con lotes de 32 muestras (`batch_size=32`), reservando el 20% de los datos para validación (`validation_split=0.2`).

El parámetro `callbacks` aplica el `early_stopping` definido, y `verbose=1` permite visualizar el progreso del entrenamiento en consola. En resumen, el modelo entrenará hasta que no haya mejoras en la validación durante 10 iteraciones consecutivas, evitando así entrenamientos innecesarios y conservando los mejores resultados.

In [9]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - Precision: 0.6319 - Recall: 0.8946 - accuracy: 0.5973 - loss: 0.7782 - val_Precision: 0.7969 - val_Recall: 0.9623 - val_accuracy: 0.8352 - val_loss: 0.4848
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - Precision: 0.7753 - Recall: 0.9861 - accuracy: 0.8148 - loss: 0.4283 - val_Precision: 0.8947 - val_Recall: 0.9623 - val_accuracy: 0.9121 - val_loss: 0.3097
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - Precision: 0.9047 - Recall: 0.9856 - accuracy: 0.9233 - loss: 0.2729 - val_Precision: 0.9286 - val_Recall: 0.9811 - val_accuracy: 0.9451 - val_loss: 0.2295
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - Precision: 0.9227 - Recall: 0.9683 - accuracy: 0.9303 - loss: 0.2343 - val_Precision: 0.9455 - val_Recall: 0.9811 - val_accuracy: 0.9560 - val_loss: 0.1804
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - Precision: 0.9489 - Recall: 0.9876 - accuracy: 0.9606 - loss: 0.1569 - val_Precision: 0.9455 - val_Recall: 0

## 4. Métricas en test.

Aquí evaluamos el rendimiento del modelo entrenado (MLP) utilizando varias métricas de clasificación de `scikit-learn`. Primero, se obtienen las probabilidades de predicción con `model.predict(X_test_scaled).ravel()` y luego se convierten en clases binarias (`0` o `1`) aplicando un umbral de 0.5. Con estas predicciones se calculan diferentes métricas:

- **Accuracy**: proporción de predicciones correctas en general.  
- **Precision**: mide qué porcentaje de las predicciones positivas son realmente positivas.  
- **Recall**: mide qué porcentaje de los casos positivos fueron identificados correctamente.  
- **F1 Score**: media armónica entre precisión y recall, útil cuando las clases están desbalanceadas.  
- **AUC-ROC**: área bajo la curva ROC, indica la capacidad del modelo para diferenciar entre clases.  
- **Matriz de confusión**: muestra los aciertos y errores del modelo separados en verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.  
- **Classification report**: presenta de forma detallada precisión, recall y F1 para cada clase.  

Este bloque permite evaluar de manera completa la calidad del modelo tanto en términos globales (accuracy, AUC) como en métricas más específicas para clases positivas y negativas (precision, recall, F1).

In [10]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

y_pred_proba_mlp = model.predict(X_test_scaled).ravel() # Get probabilities for AUC-ROC
y_pred_mlp = (y_pred_proba_mlp > 0.5).astype(int) # Convert probabilities to binary predictions

# Métricas
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)
auc_mlp = roc_auc_score(y_test, y_pred_proba_mlp)
cm_mlp = confusion_matrix(y_test, y_pred_mlp)

print("\nMétricas del modelo MLP:")
print(f"Exactitud (Accuracy): {accuracy_mlp:.4f}")
print(f"Precisión: {precision_mlp:.4f}")
print(f"Recall: {recall_mlp:.4f}")
print(f"F1: {f1_mlp:.4f}")
print(f"AUC-ROC: {auc_mlp:.4f}")
print("\nMatriz de confusión:\n", cm_mlp)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_mlp))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Métricas del modelo MLP:
Exactitud (Accuracy): 0.9649
Precisión: 0.9857
Recall: 0.9583
F1: 0.9718
AUC-ROC: 0.9901

Matriz de confusión:
 [[41  1]
 [ 3 69]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        42
           1       0.99      0.96      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114



## Conclusión:

El modelo de red neuronal MLP (Multi-Layer Perceptron) logró un desempeño sobresaliente en la clasificación binaria del dataset de cáncer de mama, alcanzando altos valores de exactitud, precisión, recall y AUC-ROC. Esto indica que el modelo es capaz de diferenciar eficazmente entre tumores malignos y benignos. Sin embargo, a diferencia de Random Forest, el MLP requiere mayor ajuste de hiperparámetros y su interpretación es menos directa, ya que no proporciona información explícita sobre la importancia de las variables. A pesar de esto, el MLP demuestra ser una alternativa potente para problemas de clasificación, especialmente cuando se busca aprovechar la capacidad de modelar relaciones no lineales complejas en los datos.

# Análisis
El modelo Random Forest demostró una excelente capacidad de clasificación, con alta exactitud y AUC, destacando por su robustez y facilidad para interpretar las variables más importantes. El MLP también logró métricas elevadas, aunque puede requerir mayor ajuste y es menos interpretable. Ambos modelos son efectivos, pero Random Forest suele ser más estable y rápido de entrenar en datos tabulares como este.